In [0]:
# from keras.models import Sequential
from keras.utils import np_utils

# importing the required modules
from keras.models import Sequential
from keras.layers import Dense, Activation, convolutional, pooling, Flatten, Dropout
from keras.optimizers import Adamax, Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianNoise
from keras.utils import to_categorical
from keras.layers import Conv2D, Activation, MaxPool2D
from keras.layers import Dense, MaxPooling2D, Activation, Conv2D, Flatten, Dropout, ZeroPadding2D # import the required layers
from keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
np.random.seed(1337)

# Loading the datasets
train_data = np.loadtxt("train_data.csv", delimiter=',')
train_target = np.loadtxt("train_target.csv", delimiter=',')
test_data = np.loadtxt("test_data.csv", delimiter=',')

#Splitting data into train set and test set
x_train,x_test,y_train,y_test = train_test_split(train_data,(train_target[:, np.newaxis]), test_size=0.3, random_state=42)

print('Before pre-processing, train data size: ', x_train.shape)
print('Before pre-processing, test data size: ', x_test.shape)

print('Before pre-processing, train target size: ', y_train.shape)
print('Before pre-processing, test target size: ', y_test.shape)


# Reshaping the data for matching the given images where 48,48 means image of size 48*48
train_data = train_data.reshape (-1,48,48,1)
test_data = test_data.reshape (-1,48,48,1)
train_target = np_utils.to_categorical (train_target, 3)

#Converting the target from scalar to vector with 3 components
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

# create a Sequential model
model = Sequential()
    
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01))) # adding a Conv2D input layer with 32 layers and input_shape of (48,48,1) and a kernel regularizer
model.add(Conv2D(32, (3, 3), padding='same',activation='relu')) # creating a Conv2D layer with 32 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1))) # creating a MaxPooling2D layer with a pool size of (2,2) and strides of (1,1)
model.add(Dropout(0.5)) # adding a Dropout layer with a probability of 0.5 for preventing overfitting
    
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(MaxPooling2D(pool_size=(2,2))) # creating a MaxPooling2D layer with a pool size of (2,2) 
model.add(Dropout(0.5)) # adding a Dropout layer with a probability of 0.5 for preventing overfitting
    
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # creating a Conv2D layer with 64 filters
model.add(BatchNormalization()) # Using batch normalization for speeding up the training process
model.add(MaxPooling2D(pool_size=(2,2))) # creating a MaxPooling2D layer with a pool size of (2,2) 
model.add(Dropout(0.5)) # adding a Dropout layer with a probability of 0.5 for preventing overfitting
        
model.add(Flatten())  # flattening the above layers
model.add(Dense(64, activation='relu')) # adding a final Dense hidden layer with 64 filters
model.add(Dropout(0.5)) # adding a Dropout layer with a probability of 0.5 for preventing overfitting
model.add(Dense(3, activation='softmax')) # an output layer that uses softmax for multi-class logistic regression
 
adam = Adamax(lr=0.0016, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compiling the model using Adamx 
model.compile(
    optimizer = adam,
    loss='categorical_crossentropy',
    metrics=['accuracy'] )

# Training the model
model.fit(train_data, train_target, validation_split=0.20, epochs=100, batch_size=100,verbose=2) 

# serializing model to JSON
model_json = model.to_json()

#Saving model 
#Need to save model for avoiding the loosing of the model due to memory overload
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")


from keras.models import model_from_json

# loading model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# loading weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# Compiling loaded model with adamax
adam = Adamax(lr=0.0019, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
loaded_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Evaluating
loss, accuracy = model.evaluate(train_data, train_target,verbose=2)
print('The loss on testing data', loss)
print('The accuracy on testing data', accuracy)
print('==========================================')

# Predicting
predictions = model.predict(test_data)
print(predictions)

import csv

# Creating the test_target set for the submission
output = predictions.argmax(axis=1)

# Writes prediction to a csv file
def write_csv(ar, file_name="result"):
    with open(file_name+".csv", 'w') as csvfile:
        writer = csv.writer(csvfile,delimiter=',',quoting = csv.QUOTE_NONE)
        row = ['Id','Category']
        writer.writerow(row)
        for i in range(ar.shape[0]):
            row = [i,ar[i]]  
            writer.writerow(row)
            i+=1
            
#writing prediction into a csv file            
write_csv(predictions)

f = open('Result.csv','w')
f.write('Id,Category\n')
for i in range(0, test_data.shape[0]):
    f.write(str(i) + ',' + str(output[i]) + '\n')
f.close()


Before pre-processing, train data size:  (11322, 2304)
Before pre-processing, test data size:  (4853, 2304)
Before pre-processing, train target size:  (11322, 1)
Before pre-processing, test target size:  (4853, 1)
Train on 12940 samples, validate on 3235 samples
Epoch 1/100
 - 26s - loss: 1.3886 - acc: 0.4235 - val_loss: 1.0811 - val_acc: 0.4464
Epoch 2/100
 - 23s - loss: 1.0784 - acc: 0.4472 - val_loss: 1.0547 - val_acc: 0.4522
Epoch 3/100
 - 23s - loss: 1.0544 - acc: 0.4503 - val_loss: 1.0130 - val_acc: 0.4869
Epoch 4/100
 - 23s - loss: 1.0181 - acc: 0.4672 - val_loss: 0.9499 - val_acc: 0.5666
Epoch 5/100
 - 23s - loss: 0.9844 - acc: 0.4940 - val_loss: 0.9769 - val_acc: 0.5308
Epoch 6/100
 - 23s - loss: 0.9688 - acc: 0.5008 - val_loss: 0.9257 - val_acc: 0.5607
Epoch 7/100
 - 23s - loss: 0.9416 - acc: 0.5122 - val_loss: 0.9006 - val_acc: 0.5756
Epoch 8/100
 - 23s - loss: 0.9329 - acc: 0.5236 - val_loss: 1.1317 - val_acc: 0.5138
Epoch 9/100
 - 23s - loss: 0.9164 - acc: 0.5569 - val_los